# Preprocess restriction data

In [1]:
import numpy as np
import pandas as pd

## Get updated dataset

In [2]:
restrictions_df = pd.read_csv('data/Kaggle_CountryInfo/restrictions.csv')

## Make some initial transformations towards a columnwise format

In [3]:
# Update type format
restrictions_df.loc[:,'type'] = 'Date.' + restrictions_df.loc[:,'type'].astype(str)

In [4]:
# rename last column
restrictions_df.columns = ['country_region', 'date', 'type', 'limit', 'mandatory', 'notes']

In [5]:
# Get set of columns and rows for the dates
countries = restrictions_df.country_region.unique()
dates = restrictions_df.type.unique()

In [6]:
#append extra info for the rows
cols = np.append(dates, ["limit", "mandatory", "notes"])

In [7]:
# Create new dataframe
df = pd.DataFrame(index=countries, columns=cols)

In [8]:
def agg_notes_without_nas(notes):
    result = []
    for note in notes:
        if note is not np.nan:
            result.append(note)
    if len(result) > 0:
        return ",".join(result)
    else: return np.nan

In [9]:
# Get Dates data
tmp_dates = restrictions_df.pivot_table(index="country_region", columns="type", values="date", aggfunc=lambda x: x)

In [10]:
df.loc[:, tmp_dates.columns] = tmp_dates.values

In [11]:
limits = restrictions_df.groupby("country_region").limit.sum()
mandatory = restrictions_df.groupby("country_region").mandatory.agg(agg_notes_without_nas)
notes = restrictions_df.groupby("country_region").notes.agg(agg_notes_without_nas)

In [12]:
df.loc[:, "limit"] = limits.values
df.loc[:, "mandatory"] = mandatory.values
df.loc[:, "notes"] = notes.values

In [13]:
# eliminate NaNs
# df = df.fillna("")

In [14]:
# make country a column again
df = df.reset_index()

In [15]:
# rename columns
df.columns = ['Country.Region', 'Date.Schools', 'Date.Public Places', 'Date.Gatherings',
       'Date.Stay at Home', 'Date.Lockdown', 'Date.Non-essential',
       'Gatherings.limit', 'mandatory', 'notes']

## Add extra columns

In [16]:
df['Country.Continent'] = np.nan
df['Province.State'] = np.nan

In [17]:
# Rearange columns
df = df.loc[:, ['Country.Continent', 'Country.Region', 'Province.State', 'Date.Schools',
       'Date.Public Places', 'Date.Gatherings', 'Date.Stay at Home',
       'Date.Lockdown', 'Date.Non-essential', 'Gatherings.limit', 'mandatory',
       'notes']]

In [18]:
# Copy the Country.Region column to Province.State
df.loc[:, 'Province.State'] = df.loc[:, 'Country.Region']

## Update Restrictions dataset with new data

### Load restriction dataset

In [19]:
restrictions_colwise_df = pd.read_csv('data/Kaggle_CountryInfo/restrictions_columnwise_updated.csv')

### Merge with new data

In [20]:
def setdiff_sorted(array1,array2,assume_unique=False):
    """
    Compares 2 arrays and returns the difference
    """
    ans = np.setdiff1d(array1,array2,assume_unique).tolist()
    if assume_unique:
        return sorted(ans)
    return ans

In [21]:
# First we update the previous data
restrictions_colwise_df.update(df, overwrite=False)

In [22]:
# then we check if there are new rows and update them
new_rows = setdiff_sorted(df['Province.State'].unique(), restrictions_colwise_df['Province.State'].unique())

In [24]:
# Add new rowsupdate
for row in new_rows:
    restrictions_colwise_df = restrictions_colwise_df.append(df[df['Province.State'] == row], ignore_index=True, verify_integrity=True)

In [68]:
# Finally we sort by Province.State 
restrictions_colwise_df.sort_values(by='Country.Region', ascending=False, inplace=True)
restrictions_colwise_df = restrictions_colwise_df.reset_index(drop=True)

In [71]:
# Save new updated version
restrictions_colwise_df.to_csv("data/Kaggle_CountryInfo/restrictions_columnwise_updated.csv", index=False)

### Auxiliary methods

In [30]:
import urllib.request

In [31]:
url = 'https://raw.githubusercontent.com/grammakov/USA-cities-and-states/master/us_cities_states_counties.csv'

In [32]:
urllib.request.urlretrieve(url, 'us_cities_states_counties.csv')

('us_cities_states_counties.csv', <http.client.HTTPMessage at 0x1168a34e0>)

In [35]:
us_states = pd.read_csv('us_cities_states_counties.csv', sep='|')

In [45]:
us_states.columns

Index(['City', 'State short', 'State full', 'County', 'City alias'], dtype='object')

In [52]:
restrictions_colwise_df.loc[restrictions_colwise_df['Province.State'].isin(us_states['State full'].unique()), 'Country.Continent'] = 'North America'

In [62]:
restrictions_colwise_df = restrictions_colwise_df.reset_index(drop=True)

In [28]:
restrictions_colwise_df[restrictions_colwise_df['Country.Region'].str.contains('Ch')]

,Country.Continent,Country.Region,Province.State,Date.Schools,Date.Public Places,Date.Gatherings,Date.Stay at Home,Date.Lockdown,Date.Non-essential,Gatherings.limit,mandatory,notes
13,Asia,China,China (Wuhan),NaN,NaN,NaN,NaN,2020-01-23T00:00:00.000000000,NaN,0,Yes,"all public transport, including buses, railway..."
12,Asia,China,China (Hubei),NaN,NaN,NaN,NaN,2020-01-24T00:00:00.000000000,2020-02-13,0,Yes,12 cities in Hubei. Shut down all non-essentia...
11,Asia,China,China,2020-01-26T00:00:00.000000000,2020-01-26T00:00:00.000000000,NaN,NaN,NaN,NaN,0,"Yes,Yes",",Covered most of the country (not 100%)"


In [72]:
df[df['Country.Region'].str.contains('Cz')]

,Country.Continent,Country.Region,Province.State,Date.Schools,Date.Public Places,Date.Gatherings,Date.Stay at Home,Date.Lockdown,Date.Non-essential,Gatherings.limit,mandatory,notes
17,,Czech,Czech,3/13/2020,3/13/2020,NaN,3/17/2020,NaN,NaN,0,"Yes,Yes,Yes",3/18: everyone is required to cover their nose...


In [63]:
# index = 3

In [64]:
# restrictions_colwise_df.loc[index, :]

Country.Continent                               NaN
Country.Region                        Washington DC
Province.State                        Washington DC
Date.Schools          2020-03-16T00:00:00.000000000
Date.Public Places    2020-03-15T00:00:00.000000000
Date.Gatherings                                 NaN
Date.Stay at Home                               NaN
Date.Lockdown                                   NaN
Date.Non-essential                              NaN
Gatherings.limit                                  0
mandatory                                   Yes,Yes
notes                                             ,
Name: 3, dtype: object

In [65]:
# Notes, will need to rename country_region to Province.State!

In [67]:
# restrictions_colwise_df.loc[index, 'Country.Region'] = 'US'

In [68]:
# df = df.append({'Country.Continent':'Middle East', 
#            'Country.Region':'Completely Made Up', 
#            'Province.State':'', 
#            'Date.Schools':'', 
#            'Date.Public Places':'', 
#            'Date.Gatherings':'', 
#            'Date.Stay at Home':'', 
#            'Date.Lockdown':'2020-3-15', 
#            'Date.Non-essential':'', 
#            'Gatherings.limit':0, 
#            'mandatory':'Yes', 
#            'notes':''}, ignore_index=True)

In [69]:
# df.loc[df["Province.State"] == '', 'Province.State'] = df.loc[df["Province.State"] == '', 'Country.Region']